In [74]:
import requests
import hashlib
import json
from openai import OpenAI
from dotenv import load_dotenv
from tqdm.auto import tqdm
import pickle


In [51]:
# load json data directly from the url 
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url, timeout=10)
documents_raw = docs_response.json()

# rearrange data a bit (add course type to each faq) LLMS likes data in the same level
documents = []
for course_dict in documents_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course'] #adding it to every faq
        documents.append(doc)

In [52]:
# we generate a unique ID for each FAQ
# doesnt need to be hashing, just some id that does not change if more data is added to the docs
# that said, hashing is the best way
def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [53]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [54]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [55]:
# save info with ids and better formatting to json file
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [56]:
# check that json printed fine
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


## Creating 5 faqs for each doc

In [67]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON format, as shown below, without using code blocks or any comment from you:

["question1", "question2", ..., "question5"]
""".strip()

In [58]:
# set up openai
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [68]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='llama3.1',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [64]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [69]:
# try out asking openAI to get 5 questions
doc = documents[0]
json_response = generate_questions(doc)
json.loads(json_response)


['What is the actual start date of the course?',
 'How will we be informed about when the first Office Hours live event will take place?',
 'Is it necessary to register for the course before attending?',
 'Can I get a reminder notification before the course starts?',
 'Are all the mentioned platforms (Google Calendar, Telegram channel, Slack, DataTalks.Club) required for me to join the course?']

In [71]:
results = {}


In [ ]:
# lets do it for all the documents
for doc in tqdm(documents):
    doc_id = doc['id']
    if doc_id in results:
        continue
    questions = generate_questions(doc)
    results[doc_id] = questions

# stopped running this because it takes forever on a laptop.
#  Run it again on workstation to see performance.


In [102]:
#  As for now, we just read in alekexys pickle file.
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [103]:
# debug a little because one results doc errors out in json
results['58c9f99f'] = results['58c9f99f'].replace('\\','/')
print(results['58c9f99f'])


[
"How can I resolve the Docker error 'invalid mode: //Program Files//Git//var//lib//postgresql//data'?",
"What should I do if I encounter an invalid mode error in Docker on Windows?",
"What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",
"Can you provide an example of a correct Docker mounting path for PostgreSQL data?",
"How do I correct the mounting path error in Docker for ///Program Files//Git//var//lib//postgresql//data'?"
]


In [104]:
# we load the raw data outputs in a json format
parsed_resulst = {}

for doc_id, json_questions in results.items():
    try:
        parsed_resulst[doc_id] = json.loads(json_questions)
    except:
        print(doc_id)
        print(json_questions)

In [105]:
# lets add a quick doc index to make csv creation easier (we need course name)
doc_index = {d['id']: d for d in documents}


In [106]:
# we write the results as a tuple
# tuple will ahve 5 questions, course name, and doc id

final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [107]:
# save the tuple as a csv via pandas
import pandas as pd

df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])
df.to_csv('ground-truth-data.csv', index=False)


In [108]:
!head ground-truth-data.csv


question,course,document
When does the course begin?,data-engineering-zoomcamp,c02e79ef
How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
Where can I find the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
How do I check the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
Where are the course prerequisites listed?,data-engineering-zoomcamp,1f6520ca
What are the requirements for joining this course?,data-engineering-zoomcamp,1f6520ca
